In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/stockmarket-sentiment-dataset/stock_data.csv


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/stockmarket-sentiment-dataset/stock_data.csv')

In [3]:
df.dropna(inplace = True)

In [4]:
import re
import nltk
from nltk.corpus import stopwords

def preprocessing_text(text):
    text = text.lower()
    text  = re.sub(r'\[.*?\]','',text)
    text  = re.sub(r'<.*?>','',text)
    text  = re.sub(r'https+//S+ | www\.S+','',text)
    text = re.sub(r'[^a-zA-Z0-9\s]','',text)
    #text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Text'] = df['Text'].apply(preprocessing_text)

In [5]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def lemm(text):
    text  = text.split(' ')
    text = [wnl.lemmatize(word) for word in text if word not in  stop_words]
    text = ' '.join(text)
    return text

df['Text'] = df['Text'].apply(lemm)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token = 'oov')
tokenizer.fit_on_texts(df['Text'])

2025-06-02 19:19:59.313637: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748891999.547816      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748891999.615069      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
len(tokenizer.word_index)

10745

In [8]:
sequences = tokenizer.texts_to_sequences(df['Text'])

In [9]:
max_len = max([len(seq) for seq in sequences])

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequences = pad_sequences(sequences , maxlen = max_len , padding = 'post' )

In [11]:
df['Sentiment_2'] = (df['Sentiment'] + 1) // 2
X = padded_sequences
y  = df['Sentiment_2'].values




In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Embedding , Bidirectional, Dropout

In [14]:
model = Sequential ([
    Embedding(10746 , 128, input_length = 24),
    LSTM(8),
#    Dense(32, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation  = 'sigmoid')
])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-06-02 19:20:13.538867: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
model.compile(optimizer= 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [16]:
history = model.fit(X_train , y_train , epochs= 5 , validation_split = 0.2)

Epoch 1/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.6302 - loss: 0.6670 - val_accuracy: 0.6268 - val_loss: 0.6651
Epoch 2/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6415 - loss: 0.6538 - val_accuracy: 0.7584 - val_loss: 0.5479
Epoch 3/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8466 - loss: 0.4157 - val_accuracy: 0.7551 - val_loss: 0.5946
Epoch 4/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9418 - loss: 0.2355 - val_accuracy: 0.7422 - val_loss: 0.7767
Epoch 5/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9728 - loss: 0.1519 - val_accuracy: 0.7271 - val_loss: 0.8977


In [17]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
              precision    recall  f1-score   support

           0       0.59      0.77      0.67       427
           1       0.84      0.69      0.76       732

    accuracy                           0.72      1159
   macro avg       0.72      0.73      0.71      1159
weighted avg       0.75      0.72      0.73      1159

